In [ ]:
import os
from glob import glob

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual

In [ ]:
from decode_trf import decode_trf, hash_file

from mosaiq_connection import mosaiq_connect
from mosaiq_field_export import (
    DeliveryData, get_bipolar_msq_data, use_mlc_missing_byte_workaround)
use_mlc_missing_byte_workaround()

In [ ]:
config = {
    "linac_logfile_data_directory": "S:\\Physics\\Programming\\data\\LinacLogFiles",
    "machine_map": {
        "2619": {
            "centre": "rccc",
            "type": "elekta-agility"
        },
        "2694": {
            "centre": "rccc",
            "type": "elekta-agility"
        },
        "4299": {
            "centre": "nbccc",
            "type": "elekta-agility"
        }
    },
    "machine_types": {
        "elekta-agility": {
            "max_leaf_gap": 400,
            "leaf_pair_widths": [
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5
            ]
        }
    },
    "centres": {
        "rccc": {
            "timezone": "Australia/Sydney",
            "ois": "mosaiq",
            "ois_specific_data": {
                "sql_user": "physics",
                "sql_server": "msqsql"
            }
        },
        "nbccc": {
            "timezone": "Australia/Sydney",
            "ois": "mosaiq",
            "ois_specific_data": {
                "sql_user": "physics",
                "sql_server": "nbccc-msq"
            }
        }
    }
}

In [ ]:
Y1_LEAF_BANK_NAMES = [
    'Y1 Leaf {}/Scaled Actual (mm)'.format(item)
    for item in range(1, 81)
]

Y2_LEAF_BANK_NAMES = [
    'Y2 Leaf {}/Scaled Actual (mm)'.format(item)
    for item in range(1, 81)
]

JAW_NAMES = [
    'X1 Diaphragm/Scaled Actual (mm)', 'X2 Diaphragm/Scaled Actual (mm)']

GANTRY_NAME = 'Step Gantry/Scaled Actual (deg)'
COLLIMATOR_NAME = 'Step Collimator/Scaled Actual (deg)'



def _pull_single_logfile(logfile_path):
    logfile_dataframe = decode_trf(logfile_path)
    raw_monitor_units = logfile_dataframe[
        'Step Dose/Actual Value (Mu)'].values.tolist()

    diff = np.append([0], np.diff(raw_monitor_units))
    diff[diff < 0] = 0

    monitor_units = np.cumsum(diff).tolist()

    gantry = logfile_dataframe[GANTRY_NAME].values.tolist()
    collimator = logfile_dataframe[COLLIMATOR_NAME].values.tolist()

    y1_bank = [
        logfile_dataframe[name].values.tolist()
        for name in Y1_LEAF_BANK_NAMES
    ]

    y2_bank = [
        logfile_dataframe[name].values.tolist()
        for name in Y2_LEAF_BANK_NAMES
    ]

    mlc = [y1_bank, y2_bank]

    jaw = [
        logfile_dataframe[name].values.tolist()
        for name in JAW_NAMES
    ]

    logfile_delivery_data = DeliveryData(
        monitor_units, gantry, collimator, mlc, jaw
    )

    return logfile_delivery_data

In [ ]:
test_files = glob(os.path.join(
    config["linac_logfile_data_directory"],
    'indexed', '*', '012194*', 'clinical', '*_VMAT', '*', '*.trf'
))

test_files

In [ ]:
delivery_data = _pull_single_logfile(test_files[0])

In [ ]:
grid_resolution = 1  # mm
mu_resolution = 1


machine_type = 'elekta-agility'

max_leaf_gap = config['machine_types'][machine_type]['max_leaf_gap']
leaf_pair_widths = np.array(config['machine_types'][machine_type]['leaf_pair_widths'])

# assert np.sum(leaf_pair_widths) == max_leaf_gap, 'non-equal max field-edge sizes not currently supported'

# num_leaf_pairs = len(leaf_pair_widths)

# assert np.all(leaf_pair_widths[0] == leaf_pair_widths), 'non-uniform leaf widths not currently supported'

# mu_density_options = {
#     'max_field_size': max_field_size,
#     'grid_resolution': grid_resolution,
#     'num_leaf_pairs': num_leaf_pairs,
#     'leaf_width': leaf_pair_widths[0],
#     'mu_resolution': mu_resolution
# }

In [ ]:
# # max_distance_from_centre = max_leaf_gap / 2
# grid_resolution = mu_density_options['grid_resolution']
# # num_leaf_pairs = mu_density_options['num_leaf_pairs']
# # leaf_width = mu_density_options['leaf_width']
# mu_resolution = mu_density_options['mu_resolution']

In [ ]:
mu_raw = np.array(delivery_data.monitor_units)
mlc_raw = np.swapaxes(delivery_data.mlc, 0, 2)
jaw_raw = np.swapaxes(delivery_data.jaw, 0, 1)

In [ ]:
leaf_x = np.arange(
    -max_leaf_gap/2,
    max_leaf_gap/2 + grid_resolution,
    grid_resolution).astype('float')

total_leaf_widths = np.sum(leaf_pair_widths)
leaf_y = np.cumsum(leaf_pair_widths) - leaf_pair_widths/2 - total_leaf_widths/2

In [ ]:
leaf_xx, leaf_yy = np.meshgrid(leaf_x, leaf_y)

In [ ]:
leaf_y

In [ ]:
np.shape(mlc_raw)[0]

In [ ]:
mlc_raw[0,:,0]

In [ ]:
def plot_mlc_jaw(i):
    plt.scatter(-mlc_raw[i,:,0], leaf_y)
    plt.scatter(mlc_raw[i,:,1], leaf_y)
    plt.scatter(0,-jaw_raw[i,0])
    plt.scatter(0,jaw_raw[i,1])

    plt.xlim([-200, 200])
    plt.ylim([-200, 200])
    
    plt.grid()
    

plot_mlc_jaw(0)

In [ ]:
interact(plot_mlc_jaw, i=(0, np.shape(mlc_raw)[0]-1))

In [ ]:
test_positions = [400, 500]

plt.figure()
plot_mlc_jaw(test_positions[0])

plt.figure()
plot_mlc_jaw(test_positions[1])

In [ ]:
leaf_xx

In [ ]:
# start_left_leaf_diffs = leaf_xx - mlc_raw[test_positions[0],:,0][:,None]
# end_left_leaf_diffs = leaf_xx - mlc_raw[test_positions[1],:,0][:,None]

# start_right_leaf_diffs = leaf_xx - mlc_raw[test_positions[0],:,1][:,None]
# end_right_leaf_diffs = leaf_xx - mlc_raw[test_positions[1],:,1][:,None]

In [ ]:
# mu = mu_raw[test_positions[1]] - mu_raw[test_positions[0]]

In [ ]:
# leaf_y[47]

In [ ]:
# start_left_leaf_diffs[47, 200]

In [ ]:
# end_left_leaf_diffs[47, 200]

In [ ]:
# left_leaf_travel_dist = np.abs(start_left_leaf_diffs - end_left_leaf_diffs)
# left_leaf_travel_dist[47, 200]

In [ ]:
# start_right_leaf_diffs[47, 200]

In [ ]:
# end_right_leaf_diffs[47, 200]

In [ ]:
# right_leaf_travel_dist = np.abs(start_right_leaf_diffs - end_right_leaf_diffs)
# right_leaf_travel_dist[47, 200]

In [ ]:
initial_leaf_grid_y_pos = leaf_y[len(leaf_y)//2]

top_grid_pos = (
    (total_leaf_widths/2 - initial_leaf_grid_y_pos) // grid_resolution *
    grid_resolution + initial_leaf_grid_y_pos)

bot_grid_pos = (
    initial_leaf_grid_y_pos -
    (total_leaf_widths/2 + initial_leaf_grid_y_pos) // grid_resolution *
    grid_resolution)

grid_y = np.arange(
    bot_grid_pos, top_grid_pos + grid_resolution, grid_resolution)

grid_leaf_map = np.argmin(np.abs(grid_y[:,None] - leaf_y[None,:]), axis=1)

In [ ]:
def calc_a_single_blocked_fraction(start_diffs, end_diffs,
                                   start_blocked, end_blocked):    
    blocked_fraction = np.ones(np.shape(start_diffs)) * np.nan
    all_open = ~start_blocked & ~end_blocked
    blocked_fraction[all_open] = 0

    all_blocked = start_blocked & end_blocked
    blocked_fraction[all_blocked] = 1

    start_blocked_fraction = np.copy(blocked_fraction)
    end_blocked_fraction = np.copy(blocked_fraction)
    
    partial_blocked = start_blocked != end_blocked
    travel = np.abs(
        start_diffs[partial_blocked] - 
        end_diffs[partial_blocked])
    
    start_partial_blocked_ref = start_blocked[partial_blocked]
    end_partial_blocked_ref = end_blocked[partial_blocked]
    
    start_blocked_fraction[partial_blocked & start_blocked] = np.abs(
        start_diffs[partial_blocked][start_partial_blocked_ref] / 
        travel[start_partial_blocked_ref]
    )
    start_blocked_fraction[partial_blocked & end_blocked] = 0
    
    end_blocked_fraction[partial_blocked & end_blocked] = np.abs(
        end_diffs[partial_blocked][end_partial_blocked_ref] / 
        travel[end_partial_blocked_ref]
    )
    end_blocked_fraction[partial_blocked & start_blocked] = 0
    
    assert np.all(~np.isnan(start_blocked_fraction))
    assert np.all(~np.isnan(end_blocked_fraction))
    
    return start_blocked_fraction, end_blocked_fraction


def calc_leaf_blocked_fractions(leaf_xx, mlc_raw, grid_leaf_map, test_positions):
    start_left_diffs = leaf_xx - -mlc_raw[test_positions[0],:,0][:,None]
    end_left_diffs = leaf_xx - -mlc_raw[test_positions[1],:,0][:,None]
    
    start_left_blocked = start_left_diffs <= 0
    end_left_blocked = end_left_diffs <= 0
    
    (
        start_left_blocked_fraction, end_left_blocked_fraction
    ) = calc_a_single_blocked_fraction(
        start_left_diffs, end_left_diffs,
        start_left_blocked, end_left_blocked)

    start_right_diffs = leaf_xx - mlc_raw[test_positions[0],:,1][:,None]
    end_right_diffs = leaf_xx - mlc_raw[test_positions[1],:,1][:,None]
    
    start_right_blocked = start_right_diffs >= 0
    end_right_blocked = end_right_diffs >= 0
    
    (
        start_right_blocked_fraction, end_right_blocked_fraction
    ) = calc_a_single_blocked_fraction(
        start_right_diffs, end_right_diffs,
        start_right_blocked, end_right_blocked)
    
    return {
        'start_left_blocked_fraction': start_left_blocked_fraction[grid_leaf_map],
        'end_left_blocked_fraction': end_left_blocked_fraction[grid_leaf_map],
        'start_right_blocked_fraction': start_right_blocked_fraction[grid_leaf_map],
        'end_right_blocked_fraction': end_right_blocked_fraction[grid_leaf_map]
    }


def calc_jaw_blocked_fraction(grid_y, jaw_raw, repeats, test_positions):
    start_top_diffs = grid_y - jaw_raw[test_positions[0],1]
    end_top_diffs = grid_y - jaw_raw[test_positions[1],1]
    
    start_top_blocked = start_top_diffs >= 0
    end_top_blocked = end_top_diffs >= 0
    
    (
        start_top_blocked_fraction, end_top_blocked_fraction
    ) = calc_a_single_blocked_fraction(
        start_top_diffs, end_top_diffs,
        start_top_blocked, end_top_blocked)
    
    start_bottom_diffs = grid_y - -jaw_raw[test_positions[0],0]
    end_bottom_diffs = grid_y - -jaw_raw[test_positions[1],0]
    
    start_bottom_blocked = start_bottom_diffs <= 0
    end_bottom_blocked = end_bottom_diffs <= 0
    
    (
        start_bottom_blocked_fraction, end_bottom_blocked_fraction
    ) = calc_a_single_blocked_fraction(
        start_bottom_diffs, end_bottom_diffs,
        start_bottom_blocked, end_bottom_blocked)
    
    return {
        'start_top_blocked_fraction': np.repeat(
            start_top_blocked_fraction[:,None], repeats, axis=1),
        'end_top_blocked_fraction': np.repeat(
            end_top_blocked_fraction[:,None], repeats, axis=1),
        'start_bottom_blocked_fraction': np.repeat(
            start_bottom_blocked_fraction[:,None], repeats, axis=1),
        'end_bottom_blocked_fraction': np.repeat(
            end_bottom_blocked_fraction[:,None], repeats, axis=1)
    }
    

leaf_blocked_fractions = calc_leaf_blocked_fractions(
    leaf_xx, mlc_raw, grid_leaf_map, test_positions)

jaw_blocked_fractions = calc_jaw_blocked_fraction(
    grid_y, jaw_raw, len(leaf_x), test_positions)

In [ ]:
plt.figure()
plot_mlc_jaw(test_positions[0])

plt.figure()
plt.pcolor(leaf_x, grid_y, jaw_blocked_fractions['start_top_blocked_fraction'])
plt.colorbar()

plt.figure()
plt.pcolor(leaf_x, grid_y, jaw_blocked_fractions['start_bottom_blocked_fraction'])
plt.colorbar()

In [ ]:
plt.figure()
plot_mlc_jaw(test_positions[1])

plt.figure()
plt.pcolor(leaf_x, grid_y, jaw_blocked_fractions['end_top_blocked_fraction'])
plt.colorbar()

plt.figure()
plt.pcolor(leaf_x, grid_y, jaw_blocked_fractions['end_bottom_blocked_fraction'])
plt.colorbar()

In [ ]:
np.shape(jaw_blocked_fractions['start_top_blocked_fraction'])

In [ ]:
np.shape(leaf_blocked_fractions['start_left_blocked_fraction'])

In [ ]:
plt.figure()
plot_mlc_jaw(test_positions[0])

plt.figure()
plt.pcolor(leaf_x, grid_y, leaf_blocked_fractions['start_left_blocked_fraction'])
plt.colorbar()
plot_mlc_jaw(test_positions[0])

plt.figure()
plt.pcolor(leaf_blocked_fractions['start_right_blocked_fraction'])
plt.colorbar()

In [ ]:
plt.figure()
plot_mlc_jaw(test_positions[1])

plt.figure()
plt.pcolor(leaf_blocked_fractions['end_left_blocked_fraction'])
plt.colorbar()

plt.figure()
plt.pcolor(leaf_blocked_fractions['end_right_blocked_fraction'])
plt.colorbar()

In [ ]:
def calc_blocked_fraction(leaf_xx, mlc_raw, grid_leaf_map, 
                          grid_y, jaw_raw, test_positions):
    
    leaf_blocked_fractions = calc_leaf_blocked_fractions(
        leaf_xx, mlc_raw, grid_leaf_map, test_positions)

    jaw_blocked_fractions = calc_jaw_blocked_fraction(
        grid_y, jaw_raw, np.shape(leaf_xx)[1], test_positions)

    start_blocked_fraction = np.max(
        [
            leaf_blocked_fractions['start_left_blocked_fraction'],
            leaf_blocked_fractions['start_right_blocked_fraction'],
            jaw_blocked_fractions['start_top_blocked_fraction'],
            jaw_blocked_fractions['start_bottom_blocked_fraction']
        ], axis=0
    )

    end_blocked_fraction = np.max(
        [
            leaf_blocked_fractions['end_left_blocked_fraction'],
            leaf_blocked_fractions['end_right_blocked_fraction'],
            jaw_blocked_fractions['end_top_blocked_fraction'],
            jaw_blocked_fractions['end_bottom_blocked_fraction']
        ], axis=0
    )

    blocked_fraction = start_blocked_fraction + end_blocked_fraction
    blocked_fraction[blocked_fraction > 1] = 1
    
    return blocked_fraction


blocked_fraction = calc_blocked_fraction(
    leaf_xx, mlc_raw, grid_leaf_map, 
    grid_y, jaw_raw, test_positions)
plt.pcolor(blocked_fraction)
plt.colorbar()

plt.figure()
plot_mlc_jaw(test_positions[0])

plt.figure()
plot_mlc_jaw(test_positions[1])

In [ ]:
np.shape(mu_raw)

In [ ]:
mu_raw[0]

In [ ]:
np.shape(mlc_raw)

In [ ]:
grid_xx, grid_yy = np.meshgrid(leaf_x, grid_y)

In [ ]:
tell_me = len(mu_raw[1::]) // 10
307 // tell_me == 307 / tell_me

In [ ]:
np.diff(mu_raw)

In [ ]:
mu_fluence = np.zeros_like(grid_xx)
tell_me = len(mu_raw[1::]) // 10


for i, mu in enumerate(np.diff(mu_raw)):
    test_positions = (i, i + 1)
    if mu != 0:
        mu_fluence += mu * (1 - calc_blocked_fraction(
            leaf_xx, mlc_raw, grid_leaf_map, 
            grid_y, jaw_raw, test_positions))
        
    if (i // tell_me == i / tell_me):
        print(i/len(mu_raw[1::]))
        
    

In [ ]:
plt.pcolormesh(grid_xx, grid_yy, mu_fluence)
plt.colorbar()

In [ ]:
mu_raw

In [ ]:
# mu_raw[1::].tolist()

In [ ]:
# np.any(end_blocked_fraction > 1)